In [ ]:
!ls "./DataPantheon/"

In [ ]:
import pandas as pd
import numpy as np
from astropy import constants as cc
from astropy import units as uu
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.rcParams['font.size'] = 18 

# Load the data from the CSV file
df = pd.read_csv("./DataPantheon/full_input.csv")  # Update the path to where your actual data file is located
df = df.sort_values("zCMB")
# Rename the first column from "Unnamed: 0" to "name"
df.rename(columns={'Unnamed: 0': 'name'}, inplace=True)

# Verify the updated column names
print(df.columns)


# Assuming df is already loaded with the data from your CSV file
# Define the absolute magnitude for Type Ia supernovae
absolute_magnitude = -19.3  # Standard absolute magnitude for Type Ia supernovae

# Calculate the luminosity distance in megaparsecs and then convert to light years
# Define the absolute magnitude for Type Ia supernovae
absolute_magnitude = -19.3
df['luminosity_distance_Mpc'] = 10 ** ((df['mB'] - absolute_magnitude + 5) / 5)/1E6 

# Convert each distance from Mpc to light-years using a list comprehension
df['luminosity_distance_lyr'] = [(d* uu.Mpc).to(uu.lyr).value for d in df['luminosity_distance_Mpc']]

# Display the first few rows of the dataframe to verify the calculations
print(df[['zCMB', 'mB', 'luminosity_distance_lyr']].head())

R0 = 14.04E9

# Ensure your DataFrame 'df' has the 'zCMB' and 'luminosity_distance_lyr' columns calculated
# Plotting redshift vs. luminosity distance in light-years
plt.figure(figsize=(10, 6))
plt.scatter(df['zCMB'], df['luminosity_distance_lyr']/R0, color='blue', s=10)  # s is the marker size
plt.xlabel('Redshift (zCMB)')
plt.ylabel('Normalized Luminosity Distance \n (fraction of $R_0$)')
plt.title('Luminosity Distance vs. Redshift')
plt.grid(True)
# plt.xscale('log')  # Optional: Use logarithmic scale for better visualization if needed
# plt.yscale('lin')
plt.savefig("./Drawing_For_Publications/PantheonSN1a.png")
plt.show()




# HU MODELING


In [ ]:
# first calculate radii for each SN1a using R(z) = 1/(1+z)
# second scale each photometric distance by R(z)**{-1.5)
R0 = 14.55E9
alpha = 1.5
df["GravitationalScalingFactor"] = (1+df.zCMB)**(-alpha)
df["HU_Distance_Pantheon"] = df.GravitationalScalingFactor *df.luminosity_distance_lyr/R0

HubbleConstant = np.round( (cc.c/(R0*uu.lyr)).to(uu.km/uu.s/uu.Mpc),2)

# Ensure your DataFrame 'df' has the 'zCMB' and 'luminosity_distance_lyr' columns calculated
# Plotting redshift vs. luminosity distance in light-years
plt.figure(figsize=(10, 6))
# Plotting the HU_Distance_Pantheon against redshift
plt.scatter(df['zCMB'], df['HU_Distance_Pantheon'], color='red', label='HU Distance (fraction of $R_0$)', s=5)  # s is the marker size

# Assuming you want to plot zCMB / (1 + zCMB) and label it properly
plt.plot(df['zCMB'], df['zCMB'] / (1 + df['zCMB']), color='blue', label='d(z) = z / (1 + z)')

plt.xlabel('Redshift (zCMB)')
plt.ylabel('Normalized Distance')
label = f'Pantheon Data for $R_0$= {R0/1E9} GLY, \n $H_0$={HubbleConstant}  and $\\alpha$={alpha} \n Corrected Normalized Distance vs. Redshift'
print(label)
plt.title(label)
plt.grid(True)
plt.legend()
# plt.xscale('log')  # Optional: Use logarithmic scale for better visualization if needed
plt.yscale('log')

# Saving the plot to a file
plt.savefig("./Drawing_For_Publications/PantheonSN1aHU.png")
plt.show()


In [ ]:
np.round( (cc.c/(14.04E9*uu.lyr)).to(uu.km/uu.s/uu.Mpc),2)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import integrate
from scipy.optimize import minimize
import astropy.constants as const
import astropy.units as uu
from astropy.cosmology import WMAP9 as cosmo
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.rcParams['font.size'] = 20 

saveplot=True

###############################################
# this choice is consistent with the current assumed age of the universe 14.04 GY and short-distance SN1a data
H_0 = 69.69411567633684 * uu.km/uu.s/uu.Mpc  
H0Value ="H06969"
fitted=1.6554778950297777
###############################################
R_0 = (const.c/H_0).to("parsec")
sn1a = pd.read_csv("./DataPantheon/full_input.csv")  # Update the path to where your actual data file is located
sn1a = df.sort_values("zCMB")
# Rename the first column from "Unnamed: 0" to "name"
sn1a.rename(columns={'Unnamed: 0': 'name'}, inplace=True)

# Verify the updated column names
print(sn1a.columns)

In [ ]:
sn1a["m"]=sn1a.mB
sn1a["z"]=sn1a.zCMB
z=sn1a.z
sn1a = sn1a [["name","z", "m"]]

# HERE WE ARE MAKING USE OF THE INTERSTELLAR EXTINCTION COEFFICIENT IN THE DATA FILE (A)!!!!!
absolute_magnitude = -19.3
sn1a["distances_obs"]=10**((sn1a.m - absolute_magnitude)/5+1)*uu.parsec
sn1a["distances_obs_normalized"]=sn1a.distances_obs/R_0
sn1a["distances_obs_comoving_normalized"]=sn1a.distances_obs_normalized/(1+z)
sn1a["modulus_distance"]=sn1a.m


sn1a=sn1a.sort_values(by=["z"])

print( "H_0=",H_0, 
      "The Universe 4D radius is {} billion light-years ".format((R_0.to("lyr").value/1E9).round(2)))


In [ ]:
# Two Plots Fitted
sn1a["alpha"]=np.pi/4-np.arcsin(1/np.sqrt(2)/(1+sn1a.z))
sn1a["HU_distances"]=1-(np.cos(sn1a.alpha)-np.sin(sn1a.alpha))
sn1a["HU_4Dradius"]= np.cos(sn1a.alpha)-np.sin(sn1a.alpha)
sn1a["GFactor"] = sn1a.HU_4Dradius**fitted #fitting
sn1a["HU_Fitted"]=sn1a.distances_obs_normalized *sn1a.GFactor
sn1a["GFactor"] = sn1a.HU_4Dradius**1.5  #parameterless prediction
sn1a["HU_Predicted"]=sn1a.distances_obs_normalized *sn1a.GFactor

print("HU Observations Normalized Distances=", sn1a.distances_obs_normalized.max(),"\n"
      "HU Normalized Distances=", sn1a.HU_distances.max())

In [ ]:
def errorf1(H0, fitted):
#     print(fitted)
    GFactor = sn1ashort.HU_4Dradius**fitted #fitting
    HH_0 = H0 * uu.km/(uu.s*uu.Mpc)
    ###############################################
    R_0 = (const.c/HH_0).to("parsec")
    sn1ashort.HU_Fitted = (sn1ashort.distances_obs/R_0) * GFactor
    err = np.abs(sn1ashort.HU_distances- sn1ashort.HU_Fitted)**0.1
    return np.sum(err)

def errorf2(fitted, H0):
    GFactor = sn1ashort.HU_4Dradius**fitted #fitting
    HH_0 = H0 * uu.km/(uu.s*uu.Mpc)
    ###############################################
    R_0 = (const.c/HH_0).to("parsec")
    sn1ashort.HU_Fitted = (sn1ashort.distances_obs/R_0) * GFactor
    err = sn1ashort.HU_distances- sn1ashort.HU_Fitted
    return np.sum(err*err)

def prediction(fitted, H0):
    GFactor = sn1ashort.HU_4Dradius**fitted #fitting
    HH_0 = H0 * uu.km/(uu.s*uu.Mpc)
    ###############################################
    R_0 = (const.c/HH_0).to("parsec")
    sn1ashort.HU_Fitted = (sn1ashort.distances_obs/R_0) * GFactor


H0 = 67.1
x0=1.6554778884148815
sn1ashort = sn1a[sn1a.z< 0.14].sort_values(by="z")
res = minimize(errorf1, x0, method='nelder-mead',args=[fitted],
               options={'xatol': 1e-8, 'disp': True})
H0=res.x[0]

In [ ]:
H0, fitted

In [ ]:
sn1ashort = sn1a[sn1a.z< 0.14].sort_values(by="z")
prediction(fitted, H0)
fig = plt.figure()
ax = plt.gca()
ax.set_ylim([0,0.15])
ax.set_xlim([0,0.15])
ax.scatter(sn1ashort.z, sn1ashort.HU_Fitted)
ax.plot(sn1ashort.z, sn1ashort.HU_distances,'b')
label = f'Pantheon Data for $R_0$= {np.round(R0,1)}, $H_0$={np.round(H0,1)}  \n Corrected Normalized Distance vs. Redshift'
ax.set_ylabel("Normalized Distance")
ax.set_xlabel("Redshift z")
ax.set_title(label)

In [ ]:

sn1ashort = sn1a[sn1a.z> 1].sort_values(by="z")
x0 = 1.0
res = minimize(errorf2, x0, method='nelder-mead',args=[H0],
               options={'xatol': 1e-8, 'disp': True})
fitted = res.x[0]
sn1ashort = sn1a[sn1a.z> 0].sort_values(by="z")
prediction(fitted,H0)
R0 = (const.c/(H0*(uu.km/(uu.s*uu.Mpc)))).to("Glyr")
fig = plt.figure()
ax = plt.gca()
# ax.scatter(sn1ashort.z, sn1ashort.HU_Fitted, 'o', c='blue', alpha=0.05, markeredgecolor='none')
ax.scatter(sn1ashort.z, sn1ashort.HU_Fitted, color='red',s=5)
ax.plot(sn1ashort.z, sn1ashort.HU_distances,'b')
label = f'Pantheon Data for $R_0$= {np.round(R0,1)}, \n $H_0$={np.round(H0,1)}  and $\\alpha$={np.round(fitted,2)} \n Corrected Normalized Distance vs. Redshift'

ax.set_xlim([-1E-2,2.5])
ax.set_ylim([1E-2,1])
ax.set_yscale('log')
ax.set_ylabel("Normalized Distance")
ax.set_xlabel("Redshift z")
plt.title(label)
plt.tight_layout(pad=2.0)

fig.savefig("./Drawing_For_Publications/Pantheon_with_H0={}_and_$\alpha=${}.png".format(np.round(H0,1),2*np.round(fitted,2)))
print(res,fitted, H0, R0)